In [1]:
from jpnotebooks.other.wql import *
import collections
import types
import datetime
import itertools
%load_ext autoreload
%autoreload 2

In [9]:
MIDNA = 'Mîdna'
SERVER = 'Turalyon'
OWNER = "nintendobratkat"
GUILD = "Synthesis"
ZONE = "Tomb of Sargeras"
MYTHIC = 5
api_key = "192b298b50ad53497d9f2b4ce7910af2"

try:
    zraw
except NameError:
    zraw = Api(api_key).zones._raw
    
def connect(api_key=api_key):
    global api
    api = Api(api_key)
    api.zones_ = Zones(api, zraw)
    return api

def pd(o):  # printdir
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o,a,"<Error>")
            if not isinstance(v, (types.FunctionType, types.MethodType)):
                print(a, "::", v)

                
# TODO: move these to wql.py once I'm confident they works properly


def _clstrfk(events, l):
    try:
        _clstrfk2(events, l)
    except ValueError:
        for c in l:
            for e in c:
                print(e.timestamp, actor(f, e))
            print()
    return l
    
def _clstrfk2(events, l):
    # debug sanity check
    seen = set()
    evd = dict.fromkeys(events, False)
    for c in l:
        for e in c:
            if evd[e] == True:
                raise ValueError("Duplicate event!")
            evd[e] = True
            
def cluster2(events, before=100, after=100):
    l = set()
    for e in events:
        l.add(tuple(events_near(events, e, before, after)))
    _clstrfk(events, l)
    return sorted(map(list,l), key=lambda c: c[0].timestamp)

def cluster(events, before=100, after=100):
    l = cluster2(events, before, after)
    l = [(c[0].timestamp, c) for c in l]
    return sorted(l, key=lambda t: t[0])
   
def hms2(t):
    t = t / 1000
    h = t // 3600
    m = (t-3600*h) // 60
    s = (t-3600*h-60*m)
    ms = (s - int(s))*1000
    return "%.2d:%.2d.%.3d" %(m,s, ms)
                
connect()
api.verbose = False

In [3]:
def deaths_near_bad(f, *bad, before=100, after=10000, cutoff=8):
    fs_deaths = 'type="death"'
    deaths = [e for e in f.events(fs_deaths, cutoff=cutoff) if e.targetID and f.actor(e.targetID).is_player()]
    fs_bads = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    bads = f.events(fs_bads, cutoff=cutoff)
    fails = set()
    for d in deaths:
        t = tuple(events_near(bads, d, before, after)), d
        fails.add(t)
    return fails

def deaths_near_bad2(f, *bad, before=100, after=10000, cutoff=8):
    fails = deaths_near_bad(f, *bad, before=before, after=after, cutoff=cutoff)
    fails2 = set()
    deaths = set()
    for ev, d in fails:
        fails2.update(ev)
        deaths.add(d)
    key = lambda e: e.timestamp
    return sorted(fails2, key=key), sorted(deaths, key=key)

def hit_by_bad(f, *bad, cutoff=8):
    fs = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    return f.events(fs, cutoff=cutoff)

In [4]:
cache = {}
def get(url):
    global cache
    if url in cache:
        return cache[url]
    rv = api.sess.request("GET", url).content.decode()
    cache[url] = rv
    return rv
def query(s):
    return pyquery.PyQuery(s)("#reports-table > tbody > tr")

In [ ]:
def throttle(t):
    def _throttle(f):
        l = time.time()
        n = l + t
        def func(*args, **kw):
            c = time.time()
            s = n-c
            if s > 0:
                time.sleep(s)
            try:
                rv = f(*args, **kw)
            finally:
                l = c
                n = c + t
            return rv
        return func
    return _throttle

@throttle(1/3)
def hf(id_):
    fights = api.fights(id_)
    if fights.find(kill=True, difficulty=5, boss=2038).fights:
        return True

In [62]:
reports = []
bosses = collections.defaultdict(list)
start = datetime.datetime.now() - datetime.timedelta(weeks=1)
for r in api.reports(GUILD, SERVER, "US").find(owner=OWNER, zone=ZONE, func=lambda r: r.start > start):
    print(r)
    fights = r.fights()
    fl = fights.find(difficulty=MYTHIC)
    for f in fl:
        bosses[f.name].append(f)
        
for v in bosses.values():
    v = sorted(set(v),key=lambda f: f.id)
bosses['Fallen Avatar']

<ReportMetadata: 'What what in the butt butt (ToS)', 'Tomb of Sargeras', 'nintendobratkat'>
<ReportMetadata: 'Smallest ToS Run EVER', 'Tomb of Sargeras', 'nintendobratkat'>
<ReportMetadata: 'Squiggly Lines (ToS)', 'Tomb of Sargeras', 'nintendobratkat'>
<ReportMetadata: 'Okay Instance Servers (ToS)', 'Tomb of Sargeras', 'nintendobratkat'>


In [37]:
energize = set()
drain = set()
keys = set()
for e in events:
    if e.type == 'energize':
        energize.update(set(e.keys()))
    elif e.type == 'drain':
        drain.update(set(e.keys()))
    else:
        raise ValueError(e.type)
    keys.update(set(e.keys()))
sorted(keys)

['ability',
 'attackPower',
 'classResources',
 'hitPoints',
 'itemLevel',
 'maxHitPoints',
 'otherResourceChange',
 'pin',
 'resolve',
 'resourceActor',
 'resourceChange',
 'resourceChangeType',
 'sourceID',
 'sourceInstance',
 'sourceIsFriendly',
 'sourceMarker',
 'spellPower',
 'targetID',
 'targetIsFriendly',
 'targetMarker',
 'timestamp',
 'type',
 'waste',
 'x',
 'y']

In [1]:
import json

def avatar_energy(f):
    # WQL doesn't return avatar's energy resources via
    # energize or drain events. The only way I've found so 
    # far to get that data is to extract the raw data used 
    # to populate the resources graph. 
    
    # manually get json data via url. "103" is the resource type. The other values should be left as is. 
    js = "/".join(map(str,(f.report_id, f.id, f.start_time, f.end_time, "103", "1", "0", "0", "-1.0.-1","0","Any","")))
    url = 'https://www.warcraftlogs.com/reports/resources_graph/'+js
    r = f.api.sess.get(url)
    r.raise_for_status()
    j = json.loads(r.content)
    
    # Data is returned for both avatar and maiden 
    for s in j['series']:
        if s['name'] == 'Fallen Avatar':
            data = s['data']
            break
    else:
        raise ValueError("Failed to find Fallen Avatar's energy")
    
    # The events we're interested in are those
    # where avatar gains more energy than normal. 
    # normal gain is 1 point of energy every 3.5 seconds
    # (alternating between 3 seconds and 4 seconds).
    # 
    evl = []
    for i in range(len(data)-1):
        x1, y1 = data[i]
        x2, y2 = data[i+1]
        if y2-y1 > 1:
            d = {"type":"energize", "timestamp":x2, "gain":(y2-y1)}
            evl.append(Struct2(d))
    return evl
    

In [120]:
for s in j['series']:
    if s['name'] == 'Fallen Avatar':
        data = s['data']

In [124]:
l = len(data)
for i in range(l-1):
    x1, y1 = data[i]
    x2, y2= data[i+1]
    el = x2-x1
    if el < 2500:
        print("Beam hit boss: %s -> %s" %(y1, y2))

Beam hit boss: 31 -> 35
Beam hit boss: 35 -> 39
Beam hit boss: 39 -> 43
Beam hit boss: 43 -> 47
Beam hit boss: 47 -> 48
Beam hit boss: 62 -> 66
Beam hit boss: 66 -> 70
Beam hit boss: 70 -> 71
Beam hit boss: 71 -> 75
Beam hit boss: 75 -> 79
Beam hit boss: 79 -> 83
Beam hit boss: 83 -> 84
Beam hit boss: 84 -> 88
Beam hit boss: 88 -> 92
Beam hit boss: 92 -> 96
Beam hit boss: 96 -> 100


In [125]:
f.id

14